### 第6週、2日目

独自のMCPサーバーとMCPクライアントを作成して使用しようとしています！

それは非常に簡単ですが、それは非常に単純ではありません。 MCPに関する興奮は、他のMCPサーバーを共有して使用するのがどれほど簡単かについてです。独自のMCPサーバーを作成するには、少しの作業が含まれます。

主に勤勉なエンジニアリングチームによって作成されたいくつかのPythonコードを確認しましょう。

accounts.py

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

In [ ]:
from accounts import Account

In [ ]:
account = Account.get("Ed")
account

In [ ]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

In [ ]:
account.report()

In [ ]:
account.list_transactions()

### 次に、MCPサーバーを書き、直接使用します！

In [ ]:
# 次に、アカウントサーバーをMCPサーバーとして使用しましょう

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [ ]:
mcp_tools

In [ ]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [ ]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


### それでは、独自のMCPクライアントを構築しましょう

In [ ]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

In [ ]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
context = await read_accounts_resource("ed")
print(context)

In [ ]:
from accounts import Account
Account.get("ed").report()

<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ assets/exercise.png" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃ff7800;">演習</h2>
            <SPAN STYLE = "Color：＃FF7800;">独自のMCPサーバーを作成します！現在の日付を返すように簡単な関数を作成し、エージェントが今日の日付を伝えることができるようにツールとして公開します。
            </span>
        </td>
    </tr>
</table>